In [2]:
import os
os.environ['AWS_ACCESS_KEY_ID']='hive'
os.environ['AWS_SECRET_ACCESS_KEY']='#202203sdf'
os.environ['AWS_REGION']='us-east-1'
           
# spark-shell --packages org.apache.iceberg:iceberg-spark-runtime-3.2_2.12:0.13.1,org.apache.iceberg:iceberg-hive-runtime:0.13.1,org.apache.hadoop:hadoop-aws:3.3.1,software.amazon.awssdk:bundle:2.15.40,software.amazon.awssdk:url-connection-client:2.15.40  --conf spark.sql.catalog.demo=org.apache.iceberg.spark.SparkCatalog    --conf spark.sql.catalog.demo.uri=thrift://localhost:9083  --conf spark.sql.catalog.demo.type=hive      --conf spark.sql.catalog.demo.io-impl=org.apache.iceberg.aws.s3.S3FileIO --conf  spark.sql.catalog.demo.warehouse=s3a://warehouse/   --conf spark.sql.catalog.demo.s3.endpoint=http://localhost:9000   --conf spark.sql.defaultCatalog=demo --conf spark.hadoop.fs.s3a.impl=org.apache.hadoop.fs.s3a.S3AFileSystem
# --conf spark.sql.catalog.spark_catalog.io-impl=org.apache.iceberg.aws.s3.S3FileIO
# --conf spark.sql.catalog.spark_catalog.s3.endpoint=http://minio:9000 
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.iceberg:iceberg-spark-runtime-3.2_2.12:0.13.1,org.apache.iceberg:iceberg-hive-runtime:0.13.1,org.apache.hadoop:hadoop-aws:3.3.1,software.amazon.awssdk:bundle:2.15.40,software.amazon.awssdk:url-connection-client:2.15.40 --conf spark.sql.catalog.spark_catalog=org.apache.iceberg.spark.SparkSessionCatalog --conf spark.sql.catalog.spark_catalog.type=hive --conf spark.sql.catalog.spark_catalog.cache-enabled=false  --conf spark.sql.catalog.spark_catalog.warehouse=s3a://hive/ --conf spark.hadoop.fs.s3a.impl=org.apache.hadoop.fs.s3a.S3AFileSystem --conf spark.sql.catalog.spark_catalog.uri=thrift://hive-metastore:9083 --conf spark.sql.catalog.spark_catalog.hadoop.fs.s3a.endpoint=http://minio:9000 --conf spark.sql.catalog.spark_catalog.s3a.endpoint=http://minio:9000 --conf spark.sql.extensions=org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions pyspark-shell'

from pyspark.sql import SparkSession

tableName = "tbltestpython"

# Enable SQL/DML commands and Metastore tables for the current spark session.
# We need to set the following configs



  
aws_key = 'hive'
aws_secret = '#202203sdf'
spark = SparkSession.builder.appName("quickstart_sql").master("local[*]") \
  .config('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider') \
  .config('spark.hadoop.fs.s3a.endpoint', 'http://minio:9000') \
  .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
  .config("spark.hadoop.fs.s3a.path.style.access", True) \
  .config('spark.hadoop.fs.s3a.access.key', aws_key) \
  .config('spark.hadoop.fs.s3a.secret.key',  aws_secret) \
  .config("hive.metastore.warehouse.dir", "s3a://hive/") \
  .config("spark.sql.warehouse.dir", "s3a://hive/") \
  .config("spark.hadoop.hive.metastore.warehouse.dir", "s3a://hive/") \
  .config("spark.hadoop.metastore.catalog.default", "hive") \
  .enableHiveSupport().getOrCreate()




In [4]:
spark.sql("use hive")

DataFrame[]

In [5]:
spark.catalog.currentDatabase()

'hive'

In [8]:
spark.sparkContext.getConf().getAll()

[('spark.sql.catalog.spark_catalog.warehouse', 's3a://hive/'),
 ('spark.files',
  'file:///home/jovyan/.ivy2/jars/org.apache.iceberg_iceberg-spark-runtime-3.2_2.12-0.13.1.jar,file:///home/jovyan/.ivy2/jars/org.apache.iceberg_iceberg-hive-runtime-0.13.1.jar,file:///home/jovyan/.ivy2/jars/org.apache.hadoop_hadoop-aws-3.3.1.jar,file:///home/jovyan/.ivy2/jars/software.amazon.awssdk_bundle-2.15.40.jar,file:///home/jovyan/.ivy2/jars/software.amazon.awssdk_url-connection-client-2.15.40.jar,file:///home/jovyan/.ivy2/jars/com.amazonaws_aws-java-sdk-bundle-1.11.901.jar,file:///home/jovyan/.ivy2/jars/org.wildfly.openssl_wildfly-openssl-1.0.7.Final.jar,file:///home/jovyan/.ivy2/jars/software.amazon.eventstream_eventstream-1.0.1.jar,file:///home/jovyan/.ivy2/jars/software.amazon.awssdk_utils-2.15.40.jar,file:///home/jovyan/.ivy2/jars/software.amazon.awssdk_annotations-2.15.40.jar,file:///home/jovyan/.ivy2/jars/software.amazon.awssdk_http-client-spi-2.15.40.jar,file:///home/jovyan/.ivy2/jars/org.rea

In [4]:
spark.sql("show databases").show()

+---------+
|namespace|
+---------+
|  default|
|     hive|
+---------+



In [11]:
spark.sql("show tables ").show()

+---------+--------------+-----------+
|namespace|     tableName|isTemporary|
+---------+--------------+-----------+
|     hive|       newdata|      false|
|     hive|tbltestpython2|      false|
+---------+--------------+-----------+



In [7]:
spark.sql(f"drop table if exists {tableName}")

DataFrame[]

In [ ]:
spark.sql(f"CREATE TABLE if not exists {tableName}2(id LONG) USING iceberg location 's3a://hive/{tableName}2'")

In [8]:
spark.sql(f"CREATE TABLE if not exists {tableName}2(id LONG) USING iceberg  ")

DataFrame[]

In [21]:
spark.sql(f"show create table tbltestpython ").show(1, False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|createtab_stmt                                                                                                                                                                                           |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|CREATE TABLE spark_catalog.hive.tbltestpython (\n  `id` BIGINT)\nUSING iceberg\nLOCATION 's3a://hive/tbltestpython'\nTBLPROPERTIES(\n  'current-snapshot-id' = 'none',\n  'format' = 'iceberg/parquet')\n|
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [7]:
spark.sql(f"select * from {tableName}").show()

+---+
| id|
+---+
+---+



In [4]:
spark.sql("drop table if exists newData")

DataFrame[]

In [10]:
spark.sql("CREATE TABLE if not exists newData(id LONG) USING parquet")

DataFrame[]

In [14]:
spark.sql(f"show create table {tableName}2").show(10, False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|createtab_stmt                                                                                                                                                                                              |
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|CREATE TABLE spark_catalog.hive.tbltestpython2 (\n  `id` BIGINT)\nUSING iceberg\nLOCATION 's3a://hive//tbltestpython2'\nTBLPROPERTIES(\n  'current-snapshot-id' = 'none',\n  'format' = 'iceberg/parquet')\n|
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [16]:

try:
    # Create a table
    print("############# Creating a table ###############")
    spark.sql("drop table if exists %s" % tableName)
    spark.sql(f"CREATE TABLE if not exists {tableName}(id LONG) USING iceberg location 's3a://hive/{tableName}' ")
    spark.sql("truncate table %s" % tableName)
    spark.sql("INSERT INTO %s VALUES 0, 1, 2, 3, 4" % tableName)

    # Read the table
    print("############ Reading the table ###############")
    spark.sql("SELECT * FROM %s" % tableName).show()

    # Upsert (merge) new data
    print("########### Upsert new data #############")
    spark.sql("drop table if exists newData")
    spark.sql(f"CREATE TABLE if not exists newData(id LONG) USING iceberg location 's3a://hive/{tableName}'   ")
    spark.sql("truncate table newData")
    spark.sql("INSERT INTO newData VALUES 3, 4, 5, 6")

    spark.sql('''MERGE INTO {0} USING newData
            ON {0}.id = newData.id
            WHEN MATCHED THEN
              UPDATE SET {0}.id = newData.id
            WHEN NOT MATCHED THEN INSERT *
        '''.format(tableName))

    spark.sql("SELECT * FROM %s" % tableName).show()

    # Update table data
    print("########## Overwrite the table ###########")
    spark.sql("INSERT OVERWRITE %s select * FROM (VALUES 5, 6, 7, 8, 9) x (id)" % tableName)
    spark.sql("SELECT * FROM %s" % tableName).show()

    # Update every even value by adding 100 to it
    print("########### Update to the table(add 100 to every even value) ##############")
    spark.sql("UPDATE {0} SET id = (id + 100) WHERE (id % 2 == 0)".format(tableName))
    spark.sql("SELECT * FROM %s" % tableName).show()

    # Delete every even value
    print("######### Delete every even value ##############")
    spark.sql("DELETE FROM {0} WHERE (id % 2 == 0)".format(tableName))
    spark.sql("SELECT * FROM %s" % tableName).show()

    # Read old version of data using time travel
    print("######## Read old data using time travel ############")
    df = spark.read.format("delta").option("versionAsOf", 1).table(tableName)
    df.show()

finally:
    # cleanup
    print("fin")
    # spark.sql("select * from " + tableName ).show()
    #spark.sql("DROP TABLE IF EXISTS " + tableName)
    #spark.sql("DROP TABLE IF EXISTS newData")

############# Creating a table ###############
############ Reading the table ###############
+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
+---+

########### Upsert new data #############
+---+
| id|
+---+
|  5|
|  6|
|  3|
|  4|
|  0|
|  1|
|  2|
+---+

########## Overwrite the table ###########
+---+
| id|
+---+
|  5|
|  6|
|  7|
|  8|
|  9|
+---+

########### Update to the table(add 100 to every even value) ##############
+---+
| id|
+---+
|106|
|108|
|  5|
|  7|
|  9|
+---+

######### Delete every even value ##############
+---+
| id|
+---+
|  5|
|  7|
|  9|
+---+

######## Read old data using time travel ############
+---+
| id|
+---+
|  5|
|  7|
|  9|
+---+

fin


In [5]:
spark.sql("UPDATE {0} SET id = (id + 100) WHERE (id % 2 == 1)".format(tableName))

DataFrame[]

In [6]:
spark.table(tableName).show()

+---+
| id|
+---+
|107|
|109|
|105|
+---+



DataFrame[]